# 📊 Market Sentiment and Trader Performance Analysis


This notebook analyzes the relationship between Bitcoin market sentiment (Fear/Greed) and trader performance data.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
trader_df = pd.read_csv('../data/historical_data.csv')
sentiment_df = pd.read_csv('../data/fear_greed_index.csv')


In [ ]:

# Preprocessing
trader_df['Trade Datetime'] = pd.to_datetime(trader_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
trader_df['Trade Date'] = trader_df['Trade Datetime'].dt.date
sentiment_df['Sentiment Date'] = pd.to_datetime(sentiment_df['date']).dt.date

# Merge
merged_df = pd.merge(trader_df, sentiment_df, left_on='Trade Date', right_on='Sentiment Date', how='left')
merged_df = merged_df.dropna(subset=['classification'])
merged_df['PnL per USD'] = merged_df['Closed PnL'] / merged_df['Size USD']
merged_df['Profit'] = merged_df['Closed PnL'] > 0

merged_df[['Trade Date', 'Side', 'Size USD', 'Closed PnL', 'classification']].head()


In [ ]:

# Analysis: Profit Rate and Trade Size by Sentiment
profit_rate = merged_df.groupby('classification')['Profit'].mean() * 100
average_size = merged_df.groupby('classification')['Size USD'].mean()

plt.figure(figsize=(10, 5))
sns.barplot(x=profit_rate.index, y=profit_rate.values)
plt.title("Profit Rate by Market Sentiment")
plt.ylabel("Profit Rate (%)")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 5))
sns.barplot(x=average_size.index, y=average_size.values, palette="magma")
plt.title("Average Trade Size by Market Sentiment")
plt.ylabel("Avg Trade Size (USD)")
plt.xticks(rotation=45)
plt.show()


In [ ]:

# Trade Side Distribution
side_distribution = merged_df.groupby(['classification', 'Side']).size().unstack(fill_value=0)
side_distribution.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='coolwarm')
plt.title("Trade Side Distribution by Sentiment")
plt.ylabel("Number of Trades")
plt.xticks(rotation=45)
plt.show()


### ✅ Observations:
- Greed days had higher profit rates.
- Fear days showed larger average trade sizes.
- Trade side behavior changes with sentiment.